In [3]:
# Adjacency List Representation of City Distances
CityGraph = {
    'cityA': [('cityB',10), ('cityC',5), ('cityD',1), ('cityG',100)],
    'cityB': [('cityA',10), ('cityC',20), ('cityD',2), ('cityE',24), ('cityG',12)],
    'cityC': [('cityA',5), ('cityB',20), ('cityD',18), ('cityF',14)],
    'cityD': [('cityB',2), ('cityC',18), ('cityA',1), ('cityE',3)],
    'cityE': [('cityB',24), ('cityD',3), ('cityF',6)],
    'cityF': [('cityC',14), ('cityE',6)],
    'cityG': [('cityA',100), ('cityB',12)]
}

In [33]:
class ShortestPath():
    '''
    O(V^2 + E).
    '''
    
    def __init__(self, adjcency_list_data):
        self.G = adjcency_list_data
        
    def find_min_city(self, unvisited, dist):
        '''
        O(V) search.
        '''
        min_city = None
        min_dist = 1e8
        for city in unvisited:
            if dist.get(city, 1e8) < min_dist:
                min_city = city
                min_dist = dist[city]
        return min_city, min_dist
        
    def compute_distances(self, cityX):
        '''
        compute distances and backpointers from cityX to all vertices.
        O(V^2 + E)
        '''
        unvisited = list(self.G.keys())
        dist = {cityX:0.,}
        prev = {cityX:None,}
        
        while unvisited: # O(V)
            curr_city, curr_dist = self.find_min_city(unvisited, dist)# O(V)
            unvisited.remove(curr_city)
            for neigh, neigh_dist in self.G[curr_city]: # O(E) in total
                if neigh in unvisited:
                    proposed_dist = curr_dist + neigh_dist
                    if proposed_dist < dist.get(neigh, 1e8):
                        dist[neigh] = proposed_dist
                        prev[neigh] = curr_city

        return dist, prev
        
    def query_distances(self, cityX, cityY):
        '''
        retrieve shortest path between two cities
        O(V) to retreive path
        '''
        dist, prev = self.compute_distances(cityX)
        curr_city = cityY
        shortest_path_reversed = [curr_city] 
        while prev[curr_city]: # O(N)
            shortest_path_reversed.append(prev[curr_city])
            curr_city = prev[curr_city]
        
        return dist[cityY], list(reversed(shortest_path_reversed)) # O(V)

In [47]:
SP = ShortestPath(CityGraph)
print(SP.query_distances('cityA', 'cityE'))

(4.0, ['cityA', 'cityD', 'cityE'])


In [58]:
import heapq

class ShortestPathWithPriorityQueue():
    '''
    O(VlogV + E). Not using min heap.
    '''
    
    def __init__(self, adjcency_list_data):
        self.G = adjcency_list_data
        
    def find_min_city(self, heap):
        '''
        Using priority queue(min heap), can reduce to O(logV): O(1) retrieve, O(logV) to maintain.
        '''
        return heapq.heappop(heap)
        
    def compute_distances(self, cityX):
        '''
        compute distances and backpointers from cityX to all vertices.
        '''
        unvisited = list(self.G.keys())
        dist = {cityX:0.,}
        heap = [(0., cityX)]
        prev = {cityX:None,}
        
        while unvisited: # O(V)
            curr_dist, curr_city = self.find_min_city(heap)# O(logV)
            if curr_city not in unvisited:
                continue
            unvisited.remove(curr_city)
            for neigh, neigh_dist in self.G[curr_city]: # O(E) in total
                if neigh in unvisited:
                    proposed_dist = curr_dist + neigh_dist
                    if proposed_dist < dist.get(neigh, 1e8):
                        dist[neigh] = proposed_dist
                        heapq.heappush(heap, (proposed_dist, neigh))
                        prev[neigh] = curr_city

        return dist, prev
        
    def query_distances(self, cityX, cityY):
        '''
        retrieve shortest path between two cities
        O(V) to retreive path
        '''
        dist, prev = self.compute_distances(cityX)
        curr_city = cityY
        shortest_path_reversed = [curr_city] 
        while prev[curr_city]: # O(N)
            shortest_path_reversed.append(prev[curr_city])
            curr_city = prev[curr_city]
        
        return dist[cityY], list(reversed(shortest_path_reversed)) # O(V)

In [64]:
SP = ShortestPathWithPriorityQueue(CityGraph)
print(SP.query_distances('cityA', 'cityE'))

(4.0, ['cityA', 'cityD', 'cityE'])


In [62]:
def test_Shortest_Path(SSP_impl):
    SP = SSP_impl(CityGraph)
    assert SP.query_distances('cityA', 'cityG')[0] == 15.
    assert SP.query_distances('cityA', 'cityD')[0] == 1.
    assert SP.query_distances('cityA', 'cityE')[0] == 4.
    assert SP.query_distances('cityF', 'cityE')[0] == 6.
    assert SP.query_distances('cityA', 'cityF')[0] == 10.
    assert SP.query_distances('cityF', 'cityF')[0] == 0.
    assert SP.query_distances('cityG', 'cityD')[0] == 14.
    
test_Shortest_Path(SSP_impl = ShortestPath)
test_Shortest_Path(SSP_impl = ShortestPathWithPriorityQueue)